<a href="https://colab.research.google.com/github/demekeendalie/Fine-tuning-bert-model-for-amharic-news-classification/blob/main/Amharic_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# reading the dataset from my drive using the above drive
dataframe =pd.read_excel('/content/drive/My Drive/output.xlsx')

In [4]:
dataframe.category.unique()

array(['accident', 'agriculture', 'health', 'education', 'economy',
       'politics', 'sport'], dtype=object)

In [5]:
# reading the second dataset from the drive
dataframe2=pd.read_excel('/content/drive/My Drive/newdata.xlsx')

In [6]:
dataframe2.category.unique()

array(['sport', 'economy', 'politics', 'accident', 'agriculture',
       'health', 'education'], dtype=object)

In [7]:
# merging the two dataframes
frame=[dataframe,dataframe2]
merge=pd.concat(frame)

In [8]:
print(merge)

                                              articles   category
0     ግንቦት 13/2011  በተሽከርካሪ አደጋ ድንገተኛ ህክምና የተፈቀደው ሁ...   accident
1    ሚያዝያ  25/ 2011 የሰሜን ተራሮች ብሄራዊ ፓርክን የአደጋ ስጋቶች ለ...   accident
2    በደቡብ ወሎ ዞን ኩታበር ወረዳ ዛሬ በደረሰ የትራፊክ አደጋ በ11 ሰዎች ...   accident
3    የህዝብ ተወካዮች ምክር ቤት አባላት ዛሬ ጠዋት የተፈናቀሉ ወገኖችን ለመጎ...   accident
4    በባህር ዳር ከተማ ቀበሌ 05 በእሳት አደጋ ቤትና ንብረት የወደመባቸው ሰ...   accident
..                                                 ...        ...
695  ጎባ፤ መስከረም 16/2015 (ኢዜአ) መደ ወላቡ ዩኒቨርሲቲ ተጨማሪ 13 ...  education
696  ባህር ዳር ሰኔ 7/2010 በአማራ ክልል ከህብረተሰቡ ከደረሱት ጥቆማዎች ...  education
697  አዲስ አበባ ሰኔ 29/2010 የትምህርት ጥራት ጉድለት በአገሪቱ ላይ ዘር...  education
698  አዲስ አበባ፤ መስከረም 15/ 2016 (ኢዜአ)፦ በተባበሩት አረብ ኤምሬቶ...  education
699  አዲስ አበባ መጋቢት 4/2012(ኢዜአ) ግማሽ ቢሊዮን ብር ወጪ የተደረገበ...  education

[2181 rows x 2 columns]


In [9]:
# Viewing the names of categories in the dataset
merge.category.unique()

array(['accident', 'agriculture', 'health', 'education', 'economy',
       'politics', 'sport'], dtype=object)

In [10]:
# changing category to numeric
label=pd.Categorical(merge['category']).codes
label=pd.DataFrame(label)
merge['labels']=pd.DataFrame(label)
print(merge)

                                              articles   category  labels
0     ግንቦት 13/2011  በተሽከርካሪ አደጋ ድንገተኛ ህክምና የተፈቀደው ሁ...   accident       0
1    ሚያዝያ  25/ 2011 የሰሜን ተራሮች ብሄራዊ ፓርክን የአደጋ ስጋቶች ለ...   accident       0
2    በደቡብ ወሎ ዞን ኩታበር ወረዳ ዛሬ በደረሰ የትራፊክ አደጋ በ11 ሰዎች ...   accident       0
3    የህዝብ ተወካዮች ምክር ቤት አባላት ዛሬ ጠዋት የተፈናቀሉ ወገኖችን ለመጎ...   accident       0
4    በባህር ዳር ከተማ ቀበሌ 05 በእሳት አደጋ ቤትና ንብረት የወደመባቸው ሰ...   accident       0
..                                                 ...        ...     ...
695  ጎባ፤ መስከረም 16/2015 (ኢዜአ) መደ ወላቡ ዩኒቨርሲቲ ተጨማሪ 13 ...  education       3
696  ባህር ዳር ሰኔ 7/2010 በአማራ ክልል ከህብረተሰቡ ከደረሱት ጥቆማዎች ...  education       3
697  አዲስ አበባ ሰኔ 29/2010 የትምህርት ጥራት ጉድለት በአገሪቱ ላይ ዘር...  education       3
698  አዲስ አበባ፤ መስከረም 15/ 2016 (ኢዜአ)፦ በተባበሩት አረብ ኤምሬቶ...  education       3
699  አዲስ አበባ መጋቢት 4/2012(ኢዜአ) ግማሽ ቢሊዮን ብር ወጪ የተደረገበ...  education       3

[2181 rows x 3 columns]


In [11]:
del merge['category']

In [12]:
# mixing each categories and thier labels
dataframe=merge.sample(frac=1)

In [13]:
#normalizarion
import re
#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalization(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48

In [14]:
# apply normalization and special characters removal
dataframe['articles']=dataframe['articles'].apply(lambda x: normalization(x))

In [15]:
# special symbol removal
dataframe['articles']=dataframe['articles'].str.replace('[^\w\s]','',regex=True)

In [16]:
# removing digits from the dataset
dataframe['articles']=dataframe['articles'].str.replace('\d+','',regex=True)

In [17]:
print(dataframe)

                                              articles  labels
461  በቻይና በሌሊት ወፍ ላይ ከኢቦላ ጋር የሚመሳሰል ቫይረስ መገኘቱ ተገለፀ\...       4
438  በሰሜን ሸዋ ዞን ነፃ የአይን ሞራ ግርዶሽ ቀዶ ህክምና በዘመቻ እየተካሄደ...       4
668  በተደረገላቸው ቤተሰባዊ አቀባበል መደሰታቸውን የአምቦ ዩኒቨርሲቲ አዲስ ገ...       3
669  ጂንካ ነሀሴ ኢዜአ የጂንካ ዩኒቨርሲቲ በተከታታይ የትምህርት መርሀ ግብር ...       3
970  የመሬት ይዞታ ማረጋጋጫው የወሰን ይገባኛል ግጭትን እንዳስቀረላቸው በምእራ...       2
..                                                 ...     ...
515  ኔዘርላንድስ ለልብ ህክምና ማእከል ማቋቋሚያ የገንዘብ ድጋፍ አደረገች\n\...       4
228  የኢትዮጵያ ፖለቲካ ከሀሳባዊነት ሳይላቀቅ የኖረበትን ዘመን እየጠቀሱ የሚያ...       1
832   ነጥብ  ሚሊዮን የእርሻ ማሳዎችን በዘመናዊ የመሬት ካርታ ለማካተት እየተ...       2
62   አክሱም ግንቦት  በአክሱም ከተማ ለአስፓልት መንገድ ግንባታ ተብለው የተቆ...       0
264  በባሌ ዞን  ሺህ ሄክታር የሚጠጋ መሬት በቡና ችግኝ ተሸፍኗል\n\nጎባ ሀ...       1

[2181 rows x 2 columns]


In [18]:
# train test split
from sklearn.model_selection import train_test_split
train_val_df, test_dataset = train_test_split(dataframe, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.10, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (1569, 2)
Validation dataset shape:  (175, 2)
Testing dataset shape:  (437, 2)


In [19]:
print(dataframe.head(10))

                                               articles  labels
461   በቻይና በሌሊት ወፍ ላይ ከኢቦላ ጋር የሚመሳሰል ቫይረስ መገኘቱ ተገለፀ\...       4
438   በሰሜን ሸዋ ዞን ነፃ የአይን ሞራ ግርዶሽ ቀዶ ህክምና በዘመቻ እየተካሄደ...       4
668   በተደረገላቸው ቤተሰባዊ አቀባበል መደሰታቸውን የአምቦ ዩኒቨርሲቲ አዲስ ገ...       3
669   ጂንካ ነሀሴ ኢዜአ የጂንካ ዩኒቨርሲቲ በተከታታይ የትምህርት መርሀ ግብር ...       3
970   የመሬት ይዞታ ማረጋጋጫው የወሰን ይገባኛል ግጭትን እንዳስቀረላቸው በምእራ...       2
1013  የኢትዮጵያና ኮትዲቯር አየር መንገዶች መንገደኞችን ለመቀባበል ተስማሙ\n\...       2
320   አዲስ አበባ መጋቢት   ኤፍ ቢ ሲ በአዲስ አበባ ከተማ ትናንት ምሽት በተ...       1
670   ጅማ መጋቢት  ቀን  ኢዜአ አሳሳቢ ደረጃ ላይ የሚገኘውን የኢትዮጵያን የት...       3
767   የአዲስ አበባ ከተማ የቴክኒክና ሙያ ትምህርትና ስልጠና ቢሮ አምስት አዳዲ...       3
566   ጥር  ዋልታ በአሸባሪው የህወሀት ቡድን የወደሙ የጤና ተቋማትን በመሰነድ ...       4


In [20]:
msk = np.random.rand(len(dataframe)) < 0.8
train_dataset = dataframe[msk]
test_dataset = dataframe[~msk]

In [21]:
!pip install datasets

In [22]:
from datasets import Dataset

In [23]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)


In [24]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [25]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [26]:
#preview of the dataset after conversion
(test_dataset)

Dataset({
    features: ['articles', 'labels', '__index_level_0__'],
    num_rows: 449
})

In [27]:
(train_dataset)

Dataset({
    features: ['articles', 'labels', '__index_level_0__'],
    num_rows: 1732
})

In [28]:
(evaluation_dataset)

Dataset({
    features: ['articles', 'labels', '__index_level_0__'],
    num_rows: 175
})

In [29]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [30]:
#combine the train and test dataset into one datset
import datasets
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [31]:
main_dataset

DatasetDict({
    train: Dataset({
        features: ['articles', 'labels'],
        num_rows: 1732
    })
    test: Dataset({
        features: ['articles', 'labels'],
        num_rows: 449
    })
    evaluate: Dataset({
        features: ['articles', 'labels'],
        num_rows: 175
    })
})

In [32]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluate'].num_rows

In [33]:
from transformers import AutoTokenizer

In [34]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

In [35]:
#Have a tokenizer function that uses the tokenizer
def tokenize_function(data):
    return tokenizer(data["articles"], padding="max_length", truncation=True)

In [36]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)

Map:   0%|          | 0/1732 [00:00<?, ? examples/s]

Map:   0%|          | 0/449 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

In [37]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["articles"])

In [38]:
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm

In [39]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [40]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [41]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 175
})

In [42]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

In [43]:
#Load auto mode classifier from the pretrained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=7)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'classifier.weight', 'bert.pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
import torch

device = torch.device("cpu")

if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")
print(device)

Training on GPU
cuda:0


In [45]:
import numpy as np
from datasets import load_metric
metric = load_metric("f1","accuracy")

<ipython-input-45-c855bf27a316>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1","accuracy")


In [46]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="micro")

In [47]:
! pip install -U accelerate
! pip install -U transformers

In [48]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [49]:
import accelerate
accelerate.__version__

'0.24.1'

In [50]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [51]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   warmup_steps=500,                # number of warmup steps for learning rate
   save_steps=2000,
   eval_steps = 2000, # Evaluation and Save happens every 50 steps
   save_total_limit = 3, # Only last 5 models are saved. Older ones are deleted.
   learning_rate=1e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=64,
   num_train_epochs=10,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

In [52]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
)

In [53]:
trainer.train()

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


TrainOutput(global_step=1090, training_loss=0.6840734753039999, metrics={'train_runtime': 1727.1719, 'train_samples_per_second': 10.028, 'train_steps_per_second': 0.631, 'total_flos': 4557288059904000.0, 'train_loss': 0.6840734753039999, 'epoch': 10.0})

In [54]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.4435746967792511,
 'eval_f1': 0.8619153674832962,
 'eval_runtime': 16.4162,
 'eval_samples_per_second': 27.351,
 'eval_steps_per_second': 0.487,
 'epoch': 10.0}

In [55]:
# evaluation
outputs=trainer.predict(small_test_dataset)
y_pred = outputs.predictions.argmax(1)

In [56]:
print(y_pred)

[5 1 6 6 0 5 0 3 4 0 1 2 4 1 2 0 2 3 0 4 6 6 5 4 3 4 0 6 3 0 5 4 0 0 1 6 1
 4 4 3 1 1 3 3 6 6 6 0 3 0 5 1 5 0 1 0 6 1 4 5 0 6 2 3 2 1 1 1 1 2 0 4 4 6
 0 2 0 1 1 0 2 0 0 1 3 1 1 3 6 1 0 3 4 6 1 2 1 0 2 5 1 0 4 2 4 4 6 4 3 0 4
 4 2 5 5 1 5 4 0 4 6 5 0 2 1 3 6 5 4 4 1 1 3 0 4 4 1 4 4 6 2 4 5 4 6 6 1 4
 6 0 0 2 3 2 2 6 4 4 0 3 4 0 4 0 1 4 4 5 0 0 4 5 6 3 5 2 2 1 6 4 4 1 0 3 4
 0 0 4 2 6 5 1 0 0 3 6 0 2 1 1 1 4 4 1 6 5 6 1 2 1 0 0 0 4 6 1 4 3 1 4 5 0
 4 4 6 1 3 5 0 1 2 2 6 1 3 4 6 1 2 6 0 6 5 3 0 0 4 1 0 3 6 1 4 0 1 4 1 4 4
 3 1 2 0 3 2 6 2 6 0 4 0 1 0 6 4 4 3 0 0 2 0 5 4 3 6 3 3 0 6 0 5 2 3 1 1 4
 1 0 4 0 1 3 3 1 5 4 2 5 4 5 0 5 2 2 0 3 2 4 3 4 0 0 0 1 4 6 0 2 1 1 3 0 1
 4 1 3 3 2 5 3 1 0 5 1 5 2 0 1 6 4 6 0 4 4 0 1 1 0 4 3 6 2 1 4 1 0 0 0 5 1
 2 6 3 1 3 6 4 6 5 3 3 5 4 5 6 6 4 3 1 1 0 6 3 1 5 3 0 4 4 4 0 2 4 1 6 5 3
 6 1 0 3 2 6 1 1 1 1 4 3 1 0 1 5 2 4 3 1 3 1 6 1 2 5 3 1 1 6 1 1 5 3 4 1 4
 4 4 6 3 2]


# New Section